# 🏠 Analyse Exploratoire des Données Immobilières

Ce notebook présente une analyse exploratoire complète des données immobilières du Québec.

## 📋 Objectifs
- Comprendre la structure et la qualité des données
- Identifier les tendances et patterns dans les prix immobiliers
- Analyser les facteurs influençant les prix
- Détecter les anomalies et valeurs aberrantes
- Visualiser les données géospatiales et temporelles

## 📊 Plan d'analyse
1. **Configuration et chargement des données**
2. **Aperçu général des données**
3. **Analyse univariée**
4. **Analyse bivariée et corrélations**
5. **Analyse géospatiale**
6. **Analyse temporelle**
7. **Insights et recommandations**


## 1. 🔧 Configuration et Chargement des Données


In [2]:
# Imports essentiels
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
import folium
from folium.plugins import MarkerCluster, HeatMap, TimestampedGeoJson
from datetime import datetime
import ast

# Configuration
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

# Imports de la bibliothèque locale
from lib.db import read_mongodb_to_dataframe
from lib.mongodb_loader import MongoDBLoader
from lib.data_processors import PropertyDataProcessor
from lib import PropertyAnalyzer
from lib.eda import *

print("✅ Configuration terminée")


✅ Configuration terminée


In [3]:
# Chargement des données
try:
    # Option 1: Connexion MongoDB
    loader = MongoDBLoader("mongodb://localhost:27017/")
    if loader.connect("real_estate_db"):
        print("🔗 Connexion MongoDB réussie")
        
        # Charger un échantillon de données
        df = loader.load_sample_data(
            collection_name="properties",
            sample_size=100000,
            filters={"type": {"$regex": "triplex", "$options": "i"}, "city":{"$regex": "trois-rivières", "$options": "i"}}
        )
        
        if not df.empty:
            print(f"📊 Données chargées: {df.shape[0]} propriétés, {df.shape[1]} colonnes")
            df.to_csv("clean_data/sample_real_estate_data.csv", index=False)
            # Aperçu général avec liste des colonnes
            loader.print_data_summary(df)
        else:
            raise Exception("Aucune donnée trouvée dans MongoDB")
    else:
        raise Exception("Connexion MongoDB échouée")
        
except Exception as e:
    print(f"⚠️ Erreur MongoDB: {e}")
    print("📁 Tentative de chargement depuis fichier CSV...")
    
    # Option 2: Chargement depuis fichier CSV
    import glob
    import os
    
    backup_dir = 'backup'
    if os.path.exists(backup_dir):
        file_pattern = os.path.join(backup_dir, "properties_db_*.csv")
        files = glob.glob(file_pattern)
        
        if files:
            latest_file = max(files, key=os.path.getmtime)
            print(f"📂 Chargement du fichier: {latest_file}")
            df = pd.read_csv(latest_file)
            # Aperçu détaillé de toutes les colonnes
            loader.print_detailed_columns_info(df)
        else:
            print("❌ Aucun fichier CSV trouvé")
    else:
        print("❌ Dossier backup non trouvé")

# Nettoyage initial des données
if 'df' in locals():
    # Convertir les dates
    date_columns = ['add_date', 'last_update']
    for col in date_columns:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')
    
    print("✅ Données prêtes pour l'analyse")
else:
    print("❌ Impossible de charger les données")


INFO:lib.mongodb_loader:✅ Connexion MongoDB établie: real_estate_db
INFO:lib.mongodb_loader:📊 Chargement d'un échantillon de 100000 propriétés...


🔗 Connexion MongoDB réussie


INFO:lib.mongodb_loader:✅ 172 propriétés chargées depuis properties


📊 Données chargées: 172 propriétés, 53 colonnes
⚠️ Erreur MongoDB: Cannot save file into a non-existent directory: 'clean_data'
📁 Tentative de chargement depuis fichier CSV...
❌ Aucun fichier CSV trouvé
✅ Données prêtes pour l'analyse


## 2. 📋 Aperçu Général des Données


In [18]:
# Informations générales sur le dataset
print("🔍 INFORMATIONS GÉNÉRALES")
print("=" * 50)
print(f"📊 Dimensions: {df.shape[0]:,} lignes × {df.shape[1]} colonnes")
print(f"💾 Taille mémoire: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"📅 Période: {df['add_date'].min().strftime('%Y-%m-%d')} à {df['add_date'].max().strftime('%Y-%m-%d')}" if 'add_date' in df.columns else "")

print("\n📝 APERÇU DES DONNÉES")
print("=" * 50)
display(df.head())


🔍 INFORMATIONS GÉNÉRALES
📊 Dimensions: 172 lignes × 53 colonnes
💾 Taille mémoire: 0.34 MB
📅 Période: 2023-02-16 à 2025-02-17

📝 APERÇU DES DONNÉES


,_id,price,type,address,description,img_src,link,plex-revenue,company,vendue,update_at,revenu,add_date,city,surface,construction_year,municipal_taxes,school_taxes,longitude,latitude,unites,depenses,price_assessment,nb_bathroom,nb_bedroom,plex-revenu,version,region,basement,bathrooms,bedrooms,building_style,commercial_units,created_at,extraction_metadata,full_address,image,images,location,lot_size,main_unit_details,municipal_evaluation_building,municipal_evaluation_land,municipal_evaluation_total,municipal_evaluation_year,municipal_tax,parking,potential_gross_revenue,residential_units,school_tax,updated_at,year_built,living_area
0,23981494,169000.00,Triplex,"{'street': '900 - 904, Rue Sainte-Angèle', 'lo...","Triplex à vendre à Trois-Rivières, Mauricie, 9...",https://mspublic.centris.ca/media.ashx?id=ADDD...,https://www.centris.ca/fr/triplex~a-vendre~tro...,Rev. bruts pot. : 19 260 $,Centris,True,"['30/05/2023', '25/06/2024']",19260.00,2023-05-30,Trois-Rivières,1875.00,1923.00,2500.00,2500.00,-72.54,46.35,"[{'unite': '4 ½', 'nb_unite': '3 '}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,28514812,179000.00,Triplex,"{'street': '1730 - 1734, Rue Saint-Paul', 'loc...","Triplex à vendre à Trois-Rivières, Mauricie, 1...",https://mspublic.centris.ca/media.ashx?id=ADDD...,https://www.centris.ca/fr/triplex~a-vendre~tro...,Rev. bruts pot. : 15 540 $,Centris,True,"[11/08/2024, 27/08/2024, 30/08/2024, 31/08/202...",15540.00,2024-08-11,Trois-Rivières,3257.00,1951.00,3373.00,104.00,-72.55,46.36,"[{'unite': '4 ½', 'nb_unite': '1 '}, {'unite':...",NaN,161300.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,23051811,199000.00,Triplex,"{'street': '924 - 926, Rue Sainte-Ursule', 'lo...","Triplex à vendre à Trois-Rivières, Mauricie, 9...",https://mspublic.centris.ca/media.ashx?id=ADDD...,https://www.centris.ca/fr/triplex~a-vendre~tro...,Rev. bruts pot. : 15 840 $,Centris,True,"['24/06/2023', '25/06/2024']",15840.00,2023-06-24,Trois-Rivières,2367.00,1923.00,2.00,90.00,-72.54,46.35,"[{'unite': '3 ½', 'nb_unite': '1 '}, {'unite':...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10812152,215000.00,Triplex,"{'street': '370 - 372, boulevard Sainte-Madele...","Triplex à vendre à Trois-Rivières, Mauricie, 3...",https://mspublic.centris.ca/media.ashx?id=ADDD...,https://www.centris.ca/fr/triplex~a-vendre~tro...,Rev. bruts pot. : 14 340 $,Centris,True,2025-02-25T19:47:17.171499,14340.00,2024-08-15,Trois-Rivières,1528.00,1920.00,NaN,NaN,-72.51,46.37,"[{'unite': '2 ½', 'nb_unite': '2 '}, {'unite':...",NaN,NaN,1,1,Rev. bruts pot. : 14 340 $,1.0.0,Mauricie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10757271,225000.00,Triplex,"{'street': '20 - 20B, boulevard Thibeau', 'loc...","Triplex à vendre à Trois-Rivières, Mauricie, 2...",https://mspublic.centris.ca/media.ashx?id=ADDD...,https://www.centris.ca/fr/triplex~a-vendre~tro...,Rev. bruts pot. : 19 440 $,Centris,True,"['16/02/2023', '25/06/2024', '29/06/2024', '06...",19440.00,2023-02-16,Trois-Rivières,6583.00,1913.00,3140.00,132.00,-72.53,46.37,"[{'unite': '4 ½', 'nb_unite': '1 '}, {'unite':...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# Analyse des valeurs manquantes
print("🕳️ ANALYSE DES VALEURS MANQUANTES")
print("=" * 50)

missing_data = df.isnull().sum().sort_values(ascending=False)
missing_percent = (missing_data / len(df) * 100).round(2)

missing_df = pd.DataFrame({
    'Colonne': missing_data.index,
    'Valeurs_manquantes': missing_data.values,
    'Pourcentage': missing_percent.values
})

# Afficher seulement les colonnes avec des valeurs manquantes
missing_df_filtered = missing_df[missing_df['Valeurs_manquantes'] > 0]

if len(missing_df_filtered) > 0:
    display(missing_df_filtered)
    
    # Visualisation des valeurs manquantes
    fig = px.bar(missing_df_filtered.head(15), 
                 x='Colonne', y='Pourcentage',
                 title='Top 15 - Pourcentage de valeurs manquantes par colonne',
                 color='Pourcentage',
                 color_continuous_scale='Reds')
    fig.update_layout(xaxis_tickangle=-45, height=500)
    fig.show()
else:
    print("✅ Aucune valeur manquante détectée!")


🕳️ ANALYSE DES VALEURS MANQUANTES


,Colonne,Valeurs_manquantes,Pourcentage
0,depenses,170,98.84
1,living_area,168,97.67
2,basement,157,91.28
3,price_assessment,155,90.12
4,parking,146,84.88
5,lot_size,141,81.98
6,location,140,81.40
7,bedrooms,140,81.40
8,building_style,140,81.40
9,commercial_units,140,81.40


In [25]:
df.columns

Index(['_id', 'price', 'type', 'address', 'description', 'img_src', 'link',
       'plex-revenue', 'company', 'vendue', 'update_at', 'revenu', 'add_date',
       'city', 'surface', 'construction_year', 'municipal_taxes',
       'school_taxes', 'longitude', 'latitude', 'unites', 'depenses',
       'price_assessment', 'nb_bathroom', 'nb_bedroom', 'plex-revenu',
       'version', 'region', 'basement', 'bathrooms', 'bedrooms',
       'building_style', 'commercial_units', 'created_at',
       'extraction_metadata', 'full_address', 'image', 'images', 'location',
       'lot_size', 'main_unit_details', 'municipal_evaluation_building',
       'municipal_evaluation_land', 'municipal_evaluation_total',
       'municipal_evaluation_year', 'municipal_tax', 'parking',
       'potential_gross_revenue', 'residential_units', 'school_tax',
       'updated_at', 'year_built', 'living_area'],
      dtype='object')

## 3. 📊 Analyse Univariée


In [20]:
# Analyse de la variable prix (principale variable d'intérêt)
print("💰 ANALYSE DÉTAILLÉE DU PRIX")
print("=" * 50)

if 'price' in df.columns:
    # Statistiques du prix
    price_stats = df['price'].describe()
    print(f"Prix moyen: {price_stats['mean']:,.0f} $")
    print(f"Prix médian: {price_stats['50%']:,.0f} $")
    print(f"Écart-type: {price_stats['std']:,.0f} $")
    print(f"Prix minimum: {price_stats['min']:,.0f} $")
    print(f"Prix maximum: {price_stats['max']:,.0f} $")
    
    # Détection des outliers
    Q1 = df['price'].quantile(0.25)
    Q3 = df['price'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[(df['price'] < lower_bound) | (df['price'] > upper_bound)]
    print(f"\n🚨 Outliers détectés: {len(outliers):,} ({len(outliers)/len(df)*100:.1f}%)")
    print(f"Borne inférieure: {lower_bound:,.0f} $")
    print(f"Borne supérieure: {upper_bound:,.0f} $")
    
    # Visualisation simple du prix
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Distribution des prix', 'Boxplot des prix')
    )
    
    # Histogramme
    fig.add_trace(
        go.Histogram(x=df['price'], nbinsx=50, name='Prix', opacity=0.7),
        row=1, col=1
    )
    
    # Boxplot
    fig.add_trace(
        go.Box(y=df['price'], name='Prix'),
        row=1, col=2
    )
    
    fig.update_layout(height=400, title_text="Analyse de la distribution des prix")
    fig.show()
else:
    print("❌ Colonne 'price' non trouvée")


💰 ANALYSE DÉTAILLÉE DU PRIX
Prix moyen: 412,126 $
Prix médian: 387,000 $
Écart-type: 150,735 $
Prix minimum: 169,000 $
Prix maximum: 999,000 $

🚨 Outliers détectés: 6 (3.5%)
Borne inférieure: 688 $
Borne supérieure: 797,988 $


## 4. 🔗 Analyse Bivariée et Corrélations


In [21]:
# Matrice de corrélation
print("🔗 MATRICE DE CORRÉLATION")
print("=" * 50)

# Sélectionner les variables numériques pour la corrélation
numeric_for_corr = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

if len(numeric_for_corr) >= 2:
    # Calculer la matrice de corrélation
    correlation_matrix = df[numeric_for_corr].corr()
    
    # Heatmap interactive avec Plotly
    fig = px.imshow(correlation_matrix, 
                    text_auto=True, 
                    aspect="auto",
                    color_continuous_scale='RdBu_r',
                    title='Matrice de corrélation des variables numériques')
    fig.update_layout(height=600)
    fig.show()
    
    # Afficher les corrélations les plus fortes avec le prix
    if 'price' in correlation_matrix.columns:
        price_corr = correlation_matrix['price'].abs().sort_values(ascending=False)[1:]  # Exclure l'auto-corrélation
        print(f"\n💰 Top 10 - Corrélations avec le prix:")
        print("-" * 40)
        for i, (var, corr) in enumerate(price_corr.head(10).items()):
            direction = "📈" if correlation_matrix['price'][var] > 0 else "📉"
            print(f"   {i+1}. {var}: {direction} {corr:.3f}")
else:
    print("❌ Pas assez de variables numériques pour calculer les corrélations")


🔗 MATRICE DE CORRÉLATION



💰 Top 10 - Corrélations avec le prix:
----------------------------------------
   1. price: 📈 1.000
   2. price_assessment: 📈 0.850
   3. municipal_evaluation_total: 📈 0.714
   4. municipal_evaluation_building: 📈 0.683
   5. revenu: 📈 0.666
   6. potential_gross_revenue: 📈 0.663
   7. school_tax: 📈 0.621
   8. municipal_evaluation_land: 📈 0.546
   9. lot_size: 📈 0.458
   10. municipal_taxes: 📈 0.436


## 5. 🗺️ Analyse Géospatiale


In [22]:
# Vérifier la disponibilité des coordonnées géographiques
print("🗺️ ANALYSE GÉOSPATIALE")
print("=" * 50)

geo_columns = ['latitude', 'longitude']
has_geo_data = all(col in df.columns for col in geo_columns)

if has_geo_data:
    # Nettoyer les données géographiques
    df_geo = df.dropna(subset=['latitude', 'longitude', 'price'])
    
    if len(df_geo) > 0:
        print(f"📍 {len(df_geo):,} propriétés avec coordonnées géographiques")
        print(f"🌐 Zone géographique:")
        print(f"   Latitude: {df_geo['latitude'].min():.3f} à {df_geo['latitude'].max():.3f}")
        print(f"   Longitude: {df_geo['longitude'].min():.3f} à {df_geo['longitude'].max():.3f}")
        
        # Carte de base
        center_lat = df_geo['latitude'].mean()
        center_lon = df_geo['longitude'].mean()
        
        print(f"\n🎯 Centre géographique: ({center_lat:.3f}, {center_lon:.3f})")
        
        # Créer une carte avec échantillonnage pour éviter trop de points
        sample_size = min(1000, len(df_geo))  # Limiter à 1000 points pour la performance
        df_sample = df_geo.sample(n=sample_size, random_state=42)
        
        print(f"\n🗺️ Création de la carte avec {sample_size} propriétés...")
        
        # Créer la carte de base
        m = folium.Map(
            location=[center_lat, center_lon],
            zoom_start=10,
            tiles='OpenStreetMap'
        )
        
        # Ajouter un cluster de marqueurs
        marker_cluster = MarkerCluster().add_to(m)
        
        # Déterminer les couleurs basées sur le prix (quartiles)
        price_q1 = df_sample['price'].quantile(0.25)
        price_q2 = df_sample['price'].quantile(0.50)
        price_q3 = df_sample['price'].quantile(0.75)
        
        def get_price_color(price):
            if price <= price_q1:
                return 'green'  # Quartile 1 - Prix bas
            elif price <= price_q2:
                return 'blue'   # Quartile 2 - Prix modéré
            elif price <= price_q3:
                return 'orange' # Quartile 3 - Prix élevé
            else:
                return 'red'    # Quartile 4 - Prix très élevé
        
        # Ajouter les marqueurs
        for idx, row in df_sample.iterrows():
            color = get_price_color(row['price'])
            
            # Construire le popup avec les informations disponibles
            popup_info = f"<b>Prix: {row['price']:,.0f} $</b><br>"
            
            # Ajouter d'autres informations si disponibles
            if 'city' in df.columns and pd.notna(row.get('city')):
                popup_info += f"Ville: {row['city']}<br>"
            if 'type' in df.columns and pd.notna(row.get('type')):
                popup_info += f"Type: {row['type']}<br>"
            if 'surface' in df.columns and pd.notna(row.get('surface')):
                popup_info += f"Surface: {row['surface']:.0f} m²<br>"
            
            # Ajouter le marqueur
            folium.Marker(
                location=[row['latitude'], row['longitude']],
                popup=folium.Popup(popup_info, max_width=200),
                tooltip=f"Prix: {row['price']:,.0f} $",
                icon=folium.Icon(color=color, icon='home')
            ).add_to(marker_cluster)
        
        # Ajouter une légende
        legend_html = f'''
        <div style="position: fixed; 
                    top: 10px; right: 10px; width: 180px; height: 120px; 
                    background-color: white; border:2px solid grey; z-index:9999; 
                    font-size:14px; padding: 10px">
        <h4>Légende des prix</h4>
        <i class="fa fa-circle" style="color:green"></i> Bas (≤ {price_q1:,.0f} $)<br>
        <i class="fa fa-circle" style="color:blue"></i> Modéré ({price_q1:,.0f} - {price_q2:,.0f} $)<br>
        <i class="fa fa-circle" style="color:orange"></i> Élevé ({price_q2:,.0f} - {price_q3:,.0f} $)<br>
        <i class="fa fa-circle" style="color:red"></i> Très élevé (> {price_q3:,.0f} $)
        </div>
        '''
        m.get_root().html.add_child(folium.Element(legend_html))
        
        # Afficher la carte
        display(m)
        
    else:
        print("❌ Aucune donnée géographique valide trouvée")
        has_geo_data = False
else:
    print("❌ Colonnes géographiques non disponibles")
    print(f"Colonnes disponibles: {df.columns.tolist()}")


🗺️ ANALYSE GÉOSPATIALE
📍 172 propriétés avec coordonnées géographiques
🌐 Zone géographique:
   Latitude: 46.298 à 46.406
   Longitude: -72.667 à -72.493

🎯 Centre géographique: (46.357, -72.545)

🗺️ Création de la carte avec 172 propriétés...


## 6. ⏰ Analyse Temporelle


In [23]:
# Analyse des tendances temporelles
print("⏰ ANALYSE TEMPORELLE")
print("=" * 50)

if 'add_date' in df.columns and 'price' in df.columns:
    # Filtrer les données avec des dates valides
    df_temporal = df.dropna(subset=['add_date', 'price'])
    
    if len(df_temporal) > 0:
        print(f"📊 {len(df_temporal):,} propriétés avec données temporelles")
        print(f"📅 Période: {df_temporal['add_date'].min().strftime('%Y-%m-%d')} à {df_temporal['add_date'].max().strftime('%Y-%m-%d')}")
        
        # Créer des colonnes temporelles
        df_temporal = df_temporal.copy()
        df_temporal['year'] = df_temporal['add_date'].dt.year
        df_temporal['month'] = df_temporal['add_date'].dt.month
        df_temporal['quarter'] = df_temporal['add_date'].dt.quarter
        df_temporal['year_month'] = df_temporal['add_date'].dt.to_period('M')
        
        # Évolution mensuelle des prix
        monthly_stats = df_temporal.groupby('year_month').agg({
            'price': ['mean', 'median', 'count']
        })
        monthly_stats.columns = ['Prix_moyen', 'Prix_médian', 'Nb_propriétés']
        monthly_stats = monthly_stats.reset_index()
        monthly_stats['date'] = monthly_stats['year_month'].astype(str)
        
        # Graphique d'évolution des prix
        fig = make_subplots(
            rows=2, cols=1,
            subplot_titles=('Évolution des prix moyens et médians', 'Nombre de propriétés ajoutées'),
            specs=[[{"secondary_y": False}], [{"secondary_y": False}]]
        )
        
        # Prix moyens et médians
        fig.add_trace(
            go.Scatter(x=monthly_stats['date'], y=monthly_stats['Prix_moyen'],
                       mode='lines+markers', name='Prix moyen', line=dict(color='blue')),
            row=1, col=1
        )
        
        fig.add_trace(
            go.Scatter(x=monthly_stats['date'], y=monthly_stats['Prix_médian'],
                       mode='lines+markers', name='Prix médian', line=dict(color='red')),
            row=1, col=1
        )
        
        # Nombre de propriétés
        fig.add_trace(
            go.Bar(x=monthly_stats['date'], y=monthly_stats['Nb_propriétés'],
                   name='Nombre de propriétés', marker=dict(color='green')),
            row=2, col=1
        )
        
        fig.update_layout(
            height=700,
            title_text="Analyse temporelle du marché immobilier",
            xaxis2_title="Période",
            yaxis_title="Prix ($)",
            yaxis2_title="Nombre de propriétés"
        )
        
        # Rotation des labels de l'axe X
        fig.update_xaxes(tickangle=45)
        
        fig.show()
        
    else:
        print("❌ Aucune donnée temporelle valide")
else:
    print("❌ Colonnes temporelles ou prix manquantes")


⏰ ANALYSE TEMPORELLE
📊 148 propriétés avec données temporelles
📅 Période: 2023-02-16 à 2025-02-17


## 7. 💡 Insights et Recommandations


In [24]:
# Génération d'insights automatiques
print("💡 INSIGHTS ET RECOMMANDATIONS")
print("=" * 50)

insights = []

# Insight 1: Taille du dataset
insights.append(f"📊 **Taille du dataset**: {len(df):,} propriétés analysées")

# Insight 2: Prix
if 'price' in df.columns:
    price_mean = df['price'].mean()
    price_median = df['price'].median()
    price_std = df['price'].std()
    
    insights.append(f"💰 **Prix moyen**: {price_mean:,.0f} $ (médian: {price_median:,.0f} $)")
    insights.append(f"📈 **Volatilité des prix**: Écart-type de {price_std:,.0f} $ ({price_std/price_mean*100:.1f}% du prix moyen)")

# Insight 3: Valeurs manquantes
missing_percent_total = (df.isnull().sum().sum() / (len(df) * len(df.columns))) * 100
if missing_percent_total > 20:
    insights.append(f"🕳️ **Qualité des données**: {missing_percent_total:.1f}% de valeurs manquantes - nettoyage recommandé")
elif missing_percent_total > 5:
    insights.append(f"🕳️ **Qualité des données**: {missing_percent_total:.1f}% de valeurs manquantes - acceptable")
else:
    insights.append(f"✅ **Excellente qualité des données**: Seulement {missing_percent_total:.1f}% de valeurs manquantes")

# Insight 4: Variables catégorielles
if 'type' in df.columns:
    type_counts = df['type'].value_counts()
    dominant_type = type_counts.index[0]
    dominant_percent = type_counts.iloc[0] / len(df) * 100
    insights.append(f"🏠 **Type dominant**: {dominant_type} ({dominant_percent:.1f}% des propriétés)")

# Insight 5: Géographie
if 'city' in df.columns:
    city_counts = df['city'].value_counts()
    top_city = city_counts.index[0]
    top_city_percent = city_counts.iloc[0] / len(df) * 100
    insights.append(f"🌍 **Ville la plus représentée**: {top_city} ({top_city_percent:.1f}% des propriétés)")
    insights.append(f"🗺️ **Diversité géographique**: {len(city_counts)} villes différentes")

# Afficher les insights
for i, insight in enumerate(insights, 1):
    print(f"{i}. {insight}")

print("\n" + "="*50)
print("🎯 RECOMMANDATIONS POUR LA SUITE")
print("="*50)

recommendations = [
    "🧹 **Nettoyage des données**: Traiter les valeurs manquantes et les outliers",
    "🔢 **Feature Engineering**: Créer de nouvelles variables (prix/m², âge du bâtiment, etc.)",
    "🤖 **Modélisation**: Développer des modèles prédictifs de prix",
    "📊 **Segmentation**: Analyser différents segments de marché",
    "🗺️ **Analyse géospatiale avancée**: Étudier l'influence des quartiers",
    "📈 **Analyse des tendances**: Approfondir l'analyse temporelle",
    "💼 **Business Intelligence**: Créer des tableaux de bord interactifs"
]

for i, rec in enumerate(recommendations, 1):
    print(f"{i}. {rec}")

print("\n✅ Analyse exploratoire terminée!")


💡 INSIGHTS ET RECOMMANDATIONS
1. 📊 **Taille du dataset**: 172 propriétés analysées
2. 💰 **Prix moyen**: 412,126 $ (médian: 387,000 $)
3. 📈 **Volatilité des prix**: Écart-type de 150,735 $ (36.6% du prix moyen)
4. 🕳️ **Qualité des données**: 54.6% de valeurs manquantes - nettoyage recommandé
5. 🏠 **Type dominant**: Triplex (81.4% des propriétés)
6. 🌍 **Ville la plus représentée**: Trois-Rivières (100.0% des propriétés)
7. 🗺️ **Diversité géographique**: 1 villes différentes

🎯 RECOMMANDATIONS POUR LA SUITE
1. 🧹 **Nettoyage des données**: Traiter les valeurs manquantes et les outliers
2. 🔢 **Feature Engineering**: Créer de nouvelles variables (prix/m², âge du bâtiment, etc.)
3. 🤖 **Modélisation**: Développer des modèles prédictifs de prix
4. 📊 **Segmentation**: Analyser différents segments de marché
5. 🗺️ **Analyse géospatiale avancée**: Étudier l'influence des quartiers
6. 📈 **Analyse des tendances**: Approfondir l'analyse temporelle
7. 💼 **Business Intelligence**: Créer des tableaux de 

## 📋 Résumé de l'Analyse

Cette analyse exploratoire a permis de:

### ✅ Ce qui a été accompli:
- **Chargement et validation** des données immobilières
- **Analyse de la qualité** des données (valeurs manquantes, types)
- **Exploration univariée** des variables clés
- **Analyse des corrélations** entre variables
- **Visualisation géospatiale** des propriétés
- **Analyse temporelle** des tendances de marché
- **Génération d'insights** actionables

### 🎯 Prochaines étapes recommandées:
1. **Préparation des données** pour la modélisation
2. **Développement de modèles prédictifs**
3. **Analyse de segments spécifiques** du marché
4. **Création de tableaux de bord interactifs**

### 📚 Ressources utilisées:
- Bibliothèque personnalisée `lib/` pour le traitement des données
- Visualisations interactives avec Plotly et Folium
- Analyses statistiques avec Pandas et NumPy

---
*Notebook créé pour l'analyse exploratoire des données immobilières du Québec*
